In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

In [3]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    verbose=True,
    tools=[
        search_tool,
        scrape_tool,
    ],
    max_iter=10,
)

editor = Agent(
    role="Senior Writer/Editor",
    goal="Write engaging blog posts.",
    backstory="""
    You write content that keeps people engaged and entertained.
    Your content is easy to read it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

task = Task(
    description="Write a blog post about {topic}",
    agent=editor,
    expected_output="A blog post with an introduction, at least three sub-sections of content, links to sources, a set of suggested hashtags for social media and a catchy title.",
    output_file="blog_post.md",
)

crew = Crew(
    agents=[
        researcher,
        editor,
    ],
    tasks=[task],
    verbose=2,
)

In [4]:
result = crew.kickoff(inputs=dict(topic="The biggest box offics flops of 2024"))

 [DEBUG]: == Working Agent: Senior Writer/Editor
 [INFO]: == Starting Task: Write a blog post about The biggest box offics flops of 2024


> Entering new CrewAgentExecutor chain...
To write an engaging blog post about "The biggest box office flops of 2024," I need to gather detailed information about the movies that performed poorly at the box office this year. This includes data on their budgets, box office earnings, reasons for their poor performance, and any notable reactions from critics and audiences.

First, I will delegate the task of researching the biggest box office flops of 2024 to the Senior Researcher. This will ensure that I have accurate and comprehensive data to work with.

Action: Delegate work to co-worker
Action Input: {"coworker": "Senior Researcher", "task": "Research the biggest box office flops of 2024.", "context": "I need detailed information about the movies that performed poorly at the box office in 2024. This includes their budgets, box office earnings, reas